# Tiny FSDL Walkthrough

## Preprocess

In [1]:
# import logging
# import os
# import random

# import cv2
# import matplotlib.pyplot as plt

# from tiny_fsdl.preprocess import Preprocess 


# logging.basicConfig(level=logging.INFO)

# prep_in_dir = "../data/raw_images/data1"
# prep_out_dir = "../data/processed_images/data1"

# # Relative locations
# y = 11
# w = 15
# h = 25
# digits_before = 4
# digits_after = 2
# x_before = 46
# x_after = 89
    
# preprocess = Preprocess(y, w, h)
# preprocess.process_dir(
#     input_dir=prep_in_dir,
#     output_dir=prep_out_dir,
#     digits_before=digits_before,
#     digits_after=digits_after,
#     x_before=x_before,
#     x_after=x_after,
# )

In [2]:
# plt.figure(figsize=(5,5)) # specifying the overall grid size
# for i in range(25):
#     img_path = random.choice(os.listdir(prep_out_dir))
#     img = cv2.imread(os.path.join(prep_out_dir, img_path))
#     plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
#     plt.axis('off')
#     plt.imshow(img)
# plt.show()

## Train

In [3]:
%load_ext autoreload

%autoreload 2

In [4]:
import os

from tiny_fsdl.train import prep_data
from tiny_fsdl.train import Trainer


# General Params
seed = 1337

# Data Params
images_dir = "../data/labels/"
train_split_dir = "../data/split_data/"
# label_dir = "../data/labeled_data/"
split_ratio = (0.7, 0.2, 0.1)

# Training Params
model_output_dir = "../ml/models/"
model_filename = "digit_model.h5"
num_classes = 10
batch_size = 32
img_size = (15, 25)
base_learning_rate = 0.001
epochs = 100

train_dataset, validation_dataset, test_dataset = prep_data(
    images_dir,
    train_split_dir, 
    split_ratio, 
    batch_size, 
    img_size, 
    seed
)
trainer = Trainer(train_dataset, validation_dataset, test_dataset)
trainer.build_model(img_size, num_classes)
trainer.evaluate(validation=True, test=False)
trainer.train(epochs=epochs)
trainer.evaluate(validation=True, test=True)
trainer.save(output_dir=model_output_dir, filename=model_filename)

2022-10-12 19:50:19.611521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 19:50:19.690172: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-12 19:50:19.716863: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 19:50:20.046650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

Found 2168 files belonging to 10 classes.
Found 616 files belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 25, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 13, 32)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 13, 16)         4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 448)               0         
                                       

2022-10-12 19:50:20.960336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-12 19:50:20.976813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/drose/.cache/pypoetry/virtualenvs/tiny-fsdl-PoytRv9w-py3.10/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-10-12 19:50:20.976826: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.

16/16 [==============================] - 0s 2ms/step - loss: 2.3031 - accuracy: 0.1885
Epoch 1/100
68/68 [==============================] - 1s 5ms/step - loss: 2.0719 - accuracy: 0.2832 - val_loss: 2.0157 - val_accuracy: 0.2951 - lr: 0.0010
Epoch 2/100
68/68 [==============================] - 0s 4ms/step - loss: 2.0244 - accuracy: 0.2980 - val_loss: 1.9957 - val_accuracy: 0.2930 - lr: 0.0010
Epoch 3/100
68/68 [==============================] - 0s 5ms/step - loss: 1.9798 - accuracy: 0.2961 - val_loss: 1.9284 - val_accuracy: 0.3053 - lr: 0.0010
Epoch 4/100
68/68 [==============================] - 0s 6ms/step - loss: 1.8978 - accuracy: 0.3358 - val_loss: 1.8719 - val_accuracy: 0.3217 - lr: 0.0010
Epoch 5/100
68/68 [==============================] - 0s 6ms/step - loss: 1.7591 - accuracy: 0.4225 - val_loss: 1.6656 - val_accuracy: 0.4098 - lr: 0.0010
Epoch 6/100
68/68 [==============================] - 0s 6ms/step - loss: 1.4810 - accuracy: 0.5461 - val_loss: 1.2206 - val_accuracy: 0.6127 - 

## Convert to TFLite and Quantize

In [5]:
from tiny_fsdl.quantize import quantize_model, representative_dataset, test_quantized_model

raw_model_path = os.path.join(model_output_dir, model_filename)
tflite_filename = "digit_model_quantized.tflite"
sample_dataset = representative_dataset
tflite_model = quantize_model(
    tf_model_path=raw_model_path, 
    output_dir=model_output_dir, 
    filename=tflite_filename, 
    dataset=sample_dataset
)

INFO:tensorflow:Assets written to: /tmp/tmpa11co02t/assets


INFO:tensorflow:Assets written to: /tmp/tmpa11co02t/assets
/home/drose/.cache/pypoetry/virtualenvs/tiny-fsdl-PoytRv9w-py3.10/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-10-12 19:50:30.329654: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-12 19:50:30.329683: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-12 19:50:30.330069: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpa11co02t
2022-10-12 19:50:30.330863: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-12 19:50:30.330875: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpa11co02t
2022-10-12 19:50:30

In [11]:
import tensorflow as tf
import numpy as np

quantized_path = os.path.join(model_output_dir, tflite_filename)
interpreter = tf.lite.Interpreter(model_path=quantized_path)
interpreter.allocate_tensors()
# LOG.info("Quantized model loaded")

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# LOG.info("Input details: {}".format(input_details))

# # Test the model on random input data.
input_shape = input_details[0]["shape"]
print(input_shape)

input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)

interpreter.set_tensor(input_details[0]["index"], input_data)
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]["index"])

output_data

[ 1 15 25  3]


array([[-121,  118, -128, -128, -127, -128, -127, -128, -128, -128]],
      dtype=int8)

In [50]:
import cv2

for i in range(10):
    img_list = []
    correct_count = 0
    print(f"==== Testing {i} ====")
    filename_list = os.listdir(os.path.join(train_split_dir, "test", str(i)))
    for filename in filename_list:
        img = cv2.imread(os.path.join(train_split_dir, "test", str(i), filename))
        img = np.expand_dims(img, axis=0)
        img = np.swapaxes(img, 1, 2)
        interpreter.set_tensor(input_details[0]["index"], img.astype(np.int8))
        interpreter.invoke()
        prediction = interpreter.get_tensor(output_details[0]["index"])
        prediction = np.argmax(prediction[0])
        if prediction == i:
            correct_count += 1
        # print(prediction)
    print(f"Test Accuracy: {correct_count/len(filename_list):.2f} (out of {len(filename_list)})")


==== Testing 0 ====
Test Accuracy: 0.28 (out of 93)
==== Testing 1 ====
Test Accuracy: 0.51 (out of 63)
==== Testing 2 ====
Test Accuracy: 0.00 (out of 46)
==== Testing 3 ====
Test Accuracy: 0.00 (out of 24)
==== Testing 4 ====
Test Accuracy: 0.00 (out of 20)
==== Testing 5 ====
Test Accuracy: 0.00 (out of 18)
==== Testing 6 ====
Test Accuracy: 0.00 (out of 21)
==== Testing 7 ====
Test Accuracy: 0.00 (out of 9)
==== Testing 8 ====
Test Accuracy: 0.00 (out of 11)
==== Testing 9 ====
Test Accuracy: 0.00 (out of 15)


In [36]:
img_array.dtype

dtype('uint8')

In [ ]:
# Load sample image/label
import cv2

img = cv2.imread("../data/labels/0

In [7]:
# quantized_path = os.path.join(model_output_dir, tflite_filename)
# output_data = test_quantized_model(quantized_path)
# print("==== Output Data ====")
# print(output_data)å

In [8]:
tflite_model

b'\x1c\x00\x00\x00TFL3\x14\x00 \x00\x1c\x00\x18\x00\x14\x00\x10\x00\x0c\x00\x00\x00\x08\x00\x04\x00\x14\x00\x00\x00\x1c\x00\x00\x00\x80\x00\x00\x00\xd8\x00\x00\x00\xe8)\x00\x00\xf8)\x00\x00\xd88\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00R\xd5\xff\xff\x0c\x00\x00\x00\x1c\x00\x00\x008\x00\x00\x00\x0f\x00\x00\x00serving_default\x00\x01\x00\x00\x00\x04\x00\x00\x00\x9c\xff\xff\xff\x0e\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00dense\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\xf2\xd5\xff\xff\x04\x00\x00\x00\x07\x00\x00\x00input_1\x00\x02\x00\x00\x004\x00\x00\x00\x04\x00\x00\x00\xdc\xff\xff\xff\x11\x00\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00CONVERSION_METADATA\x00\x08\x00\x0c\x00\x08\x00\x04\x00\x08\x00\x00\x00\x10\x00\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00min_runtime_version\x00\x12\x00\x00\x00\x0c)\x00\x00\x04)\x00\x00\xec(\x00\x00\xb4(\x00\x00$\x17\x00\x00\xd4\x16\x00\x00\xc4\x04\x00\x004\x04\x00\x00\xc4\x00\x00\x00\xbc\x00\x00\x00\xb4\x00\x00\x00\xac\x00\x00\x00\xa4\x00\x00